In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
class Pipeline():
    
    # Reading the input file and creating the Submission file
    def __init__(self,file,path):
        self.df = pd.read_csv(file)
        self.file_path = path
    
    # creating the DATE column
    def create_date(self):
        self.df['YEAR'].fillna(self.df['YEAR'].max(),inplace=True)
        self.df['MONTH'].fillna(self.df['MONTH'].max(),inplace=True)
        
        self.df['YEAR'] = self.df['YEAR'].astype(int)
        self.df['MONTH'] = self.df['MONTH'].astype(int)
        
        cols_to_convert = ['YEAR','MONTH','DAY_OF_WEEK']
        self.df['DATE'] = self.df[cols_to_convert].apply(lambda x: "-".join(x.values.astype(str)),axis=1)
    
    # creating the LOW and HIGH columns
    def create_low_high(self):
        self.df['LOW'] = self.df['DEP_TIME_BLK'].apply(lambda x: x.split("-")[0]).astype(str)
        self.df['HIGH'] = self.df['DEP_TIME_BLK'].apply(lambda x: x.split("-")[1]).astype(str)
    
    # creating the TIMESTAMP column
    def create_timestamp(self):
        self.df['RAW_TIME'] = self.df['LOW'].apply(lambda x: x[:2] + ":"+ x[2:])
        self.df['RAW_DATE'] = self.df['DATE'].astype(str)
        self.df['RAW_TIMESTAMP'] = self.df['RAW_DATE']+ " "+ self.df['RAW_TIME']
        self.df['TIMESTAMP'] = pd.to_datetime(self.df['RAW_TIMESTAMP'])
        self.df.drop(['RAW_TIME','RAW_DATE','RAW_TIMESTAMP'],axis=1,inplace=True)
    
    # creating the WIND_CHILL column
    def create_windchill(self):
        self.df['TMAX'].fillna(self.df.groupby('DEPARTING_AIRPORT')['TMAX'].transform('mean'),inplace=True)
        self.df['AWND'].fillna(self.df.groupby('DEPARTING_AIRPORT')['AWND'].transform('median'),inplace=True)
        self.df['WIND_CHILL'] = 35.74 + (0.6215*self.df['TMAX'])-(35.75*self.df['AWND'].pow(0.16)) + (0.4275*self.df['TMAX']*self.df['AWND'].pow(0.16))
    
    # creating the PRCP_SNOW_RATIO column
    def create_prcp_snow_ratio(self):
        self.df['PRCP'].fillna(self.df.groupby('DEPARTING_AIRPORT')['PRCP'].transform('median'),inplace=True)
        self.df['SNOW'].fillna(self.df.groupby('DEPARTING_AIRPORT')['SNOW'].transform('median'),inplace=True)
        
        # Using some conditions so as to not get error like zerodivision error etc.
        # Also if there is no precipitation and snow on a day, it doesn't make sense to calculate the ratio
        conditions = [
                        (self.df['PRCP'] == 0) & (self.df['SNOW'] == 0),
                        (self.df['PRCP'] == 0) & (self.df['SNOW'] != 0),
                        (self.df['PRCP'] != 0) & (self.df['SNOW'] == 0),
                        (self.df['PRCP'] != 0) & (self.df['SNOW'] != 0)
                     ]
        # Specifying values if the conditions are met
        values = [0, 0, 0, (self.df['PRCP']/self.df['SNOW'])]

        # Assigning value to the column based on the conditions
        self.df['PRCP_SNOW_RATIO'] = np.select(conditions, values)

        # Assigning the column to the submission dataframe.
        self.df['PRCP_SNOW_RATIO'] = self.df['PRCP_SNOW_RATIO']
        self.df['PRCP_SNOW_RATIO'] = self.df['PRCP_SNOW_RATIO'].astype('float32')
    
    # creating the PLANE_AGE_AIRLINE_AIRPORT_FLIGHTS_MONTH_RATIO column
    def create_plane_age_airline_airport_flights_month_ratio(self):
        self.df['PLANE_AGE'].fillna(self.df.groupby('CARRIER_NAME')['PLANE_AGE'].transform('median'),inplace=True)
        self.df['AIRLINE_AIRPORT_FLIGHTS_MONTH'].fillna(self.df.groupby(['CARRIER_NAME','DEPARTING_AIRPORT'])['AIRLINE_AIRPORT_FLIGHTS_MONTH'].transform('median'),inplace=True)
        self.df['AIRLINE_AIRPORT_FLIGHTS_MONTH'].fillna(self.df['AIRLINE_AIRPORT_FLIGHTS_MONTH'].median(),inplace=True)
        self.df['PLANE_AGE_AIRLINE_AIRPORT_FLIGHTS_MONTH_RATIO'] = self.df['PLANE_AGE'] / self.df['AIRLINE_AIRPORT_FLIGHTS_MONTH']
    
    # Creating the SEAT_DISTRIBUTION column
    def create_seat_distribution(self):
        self.df['NUMBER_OF_SEATS'].fillna(self.df.groupby('CARRIER_NAME')['NUMBER_OF_SEATS'].transform('median'),inplace=True)
        self.df['CONCURRENT_FLIGHTS'].fillna(self.df.groupby('DEP_TIME_BLK')['CONCURRENT_FLIGHTS'].transform('median'),inplace=True)
        self.df['SEAT_DISTRIBUTION'] = self.df['NUMBER_OF_SEATS'] / self.df['CONCURRENT_FLIGHTS']
    
    # Creating the SEAT_DISTRIBUTION_NORMALIZED column    
    def create_seat_distribution_normalized(self):
        self.min_SEAT_DISTRIBUTION = self.df['SEAT_DISTRIBUTION'].min()
        self.max_SEAT_DISTRIBUTION = self.df['SEAT_DISTRIBUTION'].max()
        self.df['SEAT_DISTRIBUTION_NORMALISED'] = (self.df['SEAT_DISTRIBUTION'] - self.min_SEAT_DISTRIBUTION) / (self.max_SEAT_DISTRIBUTION-self.min_SEAT_DISTRIBUTION)
    
    # Dropping Unnecessary Columns
    def drop_columns(self):
        self.df.drop(['AIRPLANE_ID','YEAR','MONTH','DAY_OF_WEEK','DEP_DEL15','DEP_TIME_BLK','DISTANCE_GROUP','SEGMENT_NUMBER',
                     'CONCURRENT_FLIGHTS','NUMBER_OF_SEATS','CARRIER_NAME','AIRPORT_FLIGHTS_MONTH','AIRLINE_FLIGHTS_MONTH',
                     'AIRLINE_AIRPORT_FLIGHTS_MONTH','AVG_MONTHLY_PASS_AIRPORT','AVG_MONTHLY_PASS_AIRLINE','FLT_ATTENDANTS_PER_PASS',
                     'GROUND_SERV_PER_PASS','PLANE_AGE','DEPARTING_AIRPORT','LATITUDE','LONGITUDE','PREVIOUS_AIRPORT','PRCP','SNOW',
                     'SNWD','TMAX','AWND'],axis=1,inplace=True)
    
    # Running all the class methods described above
    def create_submission(self):
        self.create_date()
        self.create_low_high()
        self.create_timestamp()
        self.create_windchill()
        self.create_prcp_snow_ratio()
        self.create_plane_age_airline_airport_flights_month_ratio()
        self.create_seat_distribution()
        self.create_seat_distribution_normalized()
        self.drop_columns()
        self.df.to_csv(self.file_path,index=None)

In [3]:
# Creaing the instance of Pipeline class
obj = Pipeline(r"I:\MachineHack\DataEngineering_Hack\Input_Data\participants_dataset_DES.csv",
               r"I:\MachineHack\DataEngineering_Hack\Output_Data\Submission_11.csv")

In [4]:
# Running the create_submission method on the Pipeline object
obj.create_submission()

In [5]:
# Score: 1.68411